In [1]:
import timm
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from munch import Munch
import yaml
import sys
from resnet_generator import Generator, Critic
from datetime import datetime
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
from transformer import Attention
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
from crd.criterion import CRDLoss
from dataset.imagenet import get_dataloader_sample
from timm.data import transforms_factory

In [2]:
# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.parallel import convert_syncbn_model
has_apex = False

In [12]:
# GPU Device
gpu_id = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 3
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
if args.distributed:
    args.num_gpu = 1
    args.device = 'cuda:%d' % args.local_rank
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl', init_method='env://', rank=args.rank, world_size=args.world_size)
    args.world_size = torch.distributed.get_world_size()
    args.rank = torch.distributed.get_rank()
    assert args.rank >= 0

In [7]:
torch.manual_seed(args.seed + args.rank)

In [8]:
# model_ns = timm.create_model('tf_efficientnet_b1_ns', pretrained=True)
# model_ns = model_ns.cuda()
# model_ns = torch.nn.DataParallel(model_ns)
# model_ns = model_ns.eval()

In [9]:
# fc_name = 'module.classifier.'
# teacher_model_weights = {}
# for name, param in model_ns.named_parameters():
#     teacher_model_weights[name] = param.detach()

In [10]:
# model_raw = timm.create_model('tf_efficientnet_b1', pretrained=True)
# model_raw = model_raw.cuda()
# model_raw = torch.nn.DataParallel(model_raw)

In [8]:
model_g = Generator(args, img_size=args.img_size, max_conv_dim=256)
# model_g = model_g.cuda()

In [13]:
torch.load('./output/model_best.pth.tar')

RuntimeError: CUDA error: out of memory

In [12]:
# model_critic = Critic(args)
# model_critic = model_critic.cuda()
# model_critic = torch.nn.DataParallel(model_critic)

In [13]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    model_raw =  amp.initialize(model_raw)
    model_ns = amp.initialize(model_ns)
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX not installed. AMP off.


In [14]:
# optionally resume from a checkpoint
resume_state = {}
resume_epoch = None
if args.resume:
    resume_state, resume_epoch = resume_checkpoint(model_g, args.resume)
if resume_state and not args.no_resume_opt:
    if 'optimizer' in resume_state:
        if args.local_rank == 0:
            logging.info('Restoring Optimizer state from checkpoint')
        optimizer.load_state_dict(resume_state['optimizer'])
    if use_amp and 'amp' in resume_state and 'load_state_dict' in amp.__dict__:
        if args.local_rank == 0:
            logging.info('Restoring NVIDIA AMP state from checkpoint')
        amp.load_state_dict(resume_state['amp'])
del resume_state

In [15]:
model_g = torch.nn.DataParallel(model_g)

In [16]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model_g, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 240, 240)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.882


In [17]:
num_aug_splits = 0

In [18]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP

In [19]:
# train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [20]:
param_count = sum([m.numel() for m in model_g.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 3662851


In [21]:
collate_fn = None
if args.prefetcher and args.mixup > 0:
    assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
    collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [22]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)

In [23]:
transform = transforms_factory.transforms_imagenet_train(img_size=args.img_size, auto_augment=False, color_jitter=args.color_jitter, interpolation=args.interpolation, re_count=args.recount, re_mode=args.remode, re_prob=args.reprob)

In [24]:
train_loader = get_dataloader_sample(transform, dataset='imagenet', batch_size=args.batch_size, is_sample=True, k=4096)

stage1 finished!
dataset initialized!
num_samples 1281167
num_class 1000


In [25]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [26]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.mixup > 0.:
    # smoothing is handled with mixup label transform
    train_loss_fn = SoftTargetCrossEntropy()
    validate_loss_fn = nn.CrossEntropyLoss()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    validate_loss_fn = nn.CrossEntropyLoss()
else:
    train_loss_fn = nn.CrossEntropyLoss()
    validate_loss_fn = train_loss_fn
    
crd_loss_fn = CRDLoss(args).cuda()
critic_loss_fn = nn.BCELoss()
real_label = 1
fake_label = 0

In [27]:
trainable_list = nn.ModuleList([])
trainable_list.append(model_g)
trainable_list.append(model_raw)
trainable_list.append(crd_loss_fn.embed_s)
trainable_list.append(crd_loss_fn.embed_t)

ModuleList(
  (0): DataParallel(
    (module): Generator(
      (from_rgb): Conv2d(3, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (encode): ModuleList(
        (0): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(68, 68, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(68, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (norm2): InstanceNorm2d(68, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (conv1x1): Conv2d(68, 136, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(136, 136, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(136, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(136, eps=1e-05

In [28]:
optimizer = create_optimizer(args, trainable_list)
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [29]:
optimizer_critic = create_optimizer(args, model_critic)
lr_scheduler_critic, _ = create_scheduler(args, optimizer_critic)

In [30]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 200


In [31]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [32]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [33]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [34]:
def train_epoch(epoch, model_g, model_raw, model_ns, model_critic, loader, optimizer, optimizer_critic, loss_fn, crd_loss_fn, critic_loss_fn, args,
               lr_scheduler=None, lr_scheduler_critic=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_crd = AverageMeter()
    losses_ce = AverageMeter()
    losses_kd = AverageMeter()
    losses_critic = AverageMeter()
    losses_g = AverageMeter()
    
    model_g.train()
    model_critic.train()
    model_raw.train()
    model_ns.eval()
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
        
    for batch_idx, data in enumerate(loader):
        inputs, target, index, contrast_idx = data
        inputs, target, index, contrast_idx = inputs.cuda(), target.cuda(), index.cuda(), contrast_idx.cuda()
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        
        g_out = model_g(inputs)
        
        inputs_z = inputs + g_out
        output, f_raw = model_raw(inputs_z)
        with torch.no_grad():
#             inputs_ns = F.interpolate(inputs, size=300, mode='bicubic')
            out_ns, f_ns = model_ns(inputs)
            out_ns = out_ns.detach()
        
        # Critic
        model_critic.train()
        model_critic.zero_grad()
        label = torch.full((f_raw.size(0), ), real_label, device='cuda')
        critic_out = model_critic(f_ns).view(-1)
        loss_critic_ns = args.lambda_critic * critic_loss_fn(critic_out, label)
        loss_critic_ns.backward()
        loss_critic_ns_out = loss_critic_ns.item()
        
        label.fill_(fake_label)
        critic_out = model_critic(f_raw.detach()).view(-1)
        loss_critic_raw = critic_loss_fn(critic_out, label)
        loss_critic_raw.backward()
        loss_critic_raw_out = args.lambda_critic * loss_critic_raw.item()
        loss_critic = loss_critic_ns_out + loss_critic_raw_out
        optimizer_critic.step()
        
        # KD
        p_s = F.log_softmax(output/args.T, dim=1)
        p_t = F.softmax(out_ns/args.T, dim=1)
        loss_kd = F.kl_div(p_s, p_t, size_average=False) * (args.T ** 2) / output.shape[0]
        
       # CE
        loss_ce = loss_fn(output, target)
        
        # CRD
        loss_crd = crd_loss_fn(f_raw, f_ns, index, contrast_idx)
        
        # G
        model_critic.eval()
        label.fill_(real_label)
        critic_g_out = model_critic(f_raw).view(-1)
        loss_g = critic_loss_fn(critic_g_out, label)
        
        # overall loss
        loss = args.lambda_kd * loss_kd + args.lambda_ce * loss_ce + args.lambda_crd * loss_crd + args.lambda_g * loss_g
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not args.distributed:
            losses_kd.update(loss_kd.item(), inputs.size(0))
            losses_ce.update(loss_ce.item(), inputs.size(0))
            losses_crd.update(loss_crd.item(), inputs.size(0))
            losses_critic.update(loss_critic, inputs.size(0))
            losses_g.update(loss_g.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))

        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                    'Loss_kd: {loss_kd.val:>9.6f} ({loss_kd.avg:>6.4f})  '
                    'Loss_ce: {loss_ce.val:>9.6f} ({loss_ce.avg:>6.4f})  '
                    'Loss_crd: {loss_crd.val:>9.6f} ({loss_crd.avg:>6.4f})  '
                    'Loss_g: {loss_g.val:>9.6f} ({loss_g.avg:>6.4f})  '
                    'Loss_critic: {loss_critic.val:>9.6f} ({loss_critic.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.7f}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
                        loss_kd=losses_kd,
                        loss_ce=losses_ce,
                        loss_crd=losses_crd,
                        loss_g=losses_g,
                        loss_critic=losses_critic,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model_raw, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)
            
        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)
            lr_scheduler_critic.step_update(num_updates=num_updates, metric=losses_critic.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()
        

    return OrderedDict([('loss', losses_m.avg)])

In [35]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
#     model_att.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()
            
            out = model(inputs)
            # synthesizing input + generator
            inputs_out = inputs + out
            # compute output
            output, _ = model_raw(inputs_out)
#             output = model_att(inputs_out, output)
            loss = criterion(output, target)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [ ]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        train_loader.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model_g, model_raw, model_ns, model_critic, train_loader, optimizer, optimizer_critic, 
        train_loss_fn, crd_loss_fn, critic_loss_fn, args,
        lr_scheduler=lr_scheduler, lr_scheduler_critic=lr_scheduler_critic, 
        saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    eval_metrics = val_epoch(model_raw, model_g, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')

    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])
        
    update_summary( 
        epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
        write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model_g, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)
        torch.save(model_raw, 'raw')
        torch.save(model_critic, 'critic')

/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


normalization constant Z_v1 is set to 2842167.2
normalization constant Z_v2 is set to 2835724.2


INFO:root:Train: 1 [   0/8374 (  0%)]  Loss: 20.373619 (20.3736)  Loss_kd:  0.532949 (0.5329)  Loss_ce:  3.193007 (3.1930)  Loss_crd: 20.723337 (20.7233)  Loss_g:  0.689934 (0.6899)  Loss_critic:  0.141794 (0.1418)  Time: 23.286s,   19.71/s  (23.286s,   19.71/s)  LR: 0.0025600  Data: 5.177 (5.177)
INFO:root:Train: 1 [  50/8374 (  1%)]  Loss: 19.057383 (18.9987)  Loss_kd:  0.389808 (0.4169)  Loss_ce:  1.907302 (2.0310)  Loss_crd: 20.686684 (20.4922)  Loss_g:  2.109257 (1.5703)  Loss_critic:  0.145763 (0.1349)  Time: 1.194s,  384.45/s  (1.683s,  272.68/s)  LR: 0.0025600  Data: 0.013 (0.114)
INFO:root:Train: 1 [ 100/8374 (  1%)]  Loss: 18.864117 (18.9549)  Loss_kd:  0.369464 (0.4002)  Loss_ce:  1.900531 (1.9848)  Loss_crd: 20.475639 (20.4802)  Loss_g:  2.136104 (1.8584)  Loss_critic:  0.159211 (0.1454)  Time: 1.181s,  388.69/s  (1.437s,  319.36/s)  LR: 0.0025600  Data: 0.013 (0.064)
INFO:root:Train: 1 [ 150/8374 (  2%)]  Loss: 19.083406 (18.9490)  Loss_kd:  0.354735 (0.3889)  Loss_ce:  2.

INFO:root:Train: 1 [1400/8374 ( 17%)]  Loss: 18.694157 (18.7735)  Loss_kd:  0.350359 (0.3582)  Loss_ce:  2.080412 (1.8983)  Loss_crd: 20.044764 (20.3737)  Loss_g:  2.275729 (2.1802)  Loss_critic:  0.224351 (0.1817)  Time: 1.179s,  389.15/s  (1.205s,  381.00/s)  LR: 0.0025600  Data: 0.013 (0.017)
INFO:root:Train: 1 [1450/8374 ( 17%)]  Loss: 18.428593 (18.7673)  Loss_kd:  0.349403 (0.3579)  Loss_ce:  1.883894 (1.8974)  Loss_crd: 19.966873 (20.3670)  Loss_g:  2.217980 (2.1841)  Loss_critic:  0.233485 (0.1833)  Time: 1.181s,  388.63/s  (1.204s,  381.17/s)  LR: 0.0025600  Data: 0.013 (0.017)
INFO:root:Train: 1 [1500/8374 ( 18%)]  Loss: 18.425110 (18.7607)  Loss_kd:  0.359863 (0.3577)  Loss_ce:  1.779275 (1.8956)  Loss_crd: 20.056633 (20.3607)  Loss_g:  2.406662 (2.1888)  Loss_critic:  0.236223 (0.1850)  Time: 1.196s,  383.73/s  (1.204s,  381.35/s)  LR: 0.0025600  Data: 0.013 (0.016)
INFO:root:Train: 1 [1550/8374 ( 19%)]  Loss: 18.780834 (18.7556)  Loss_kd:  0.365109 (0.3575)  Loss_ce:  1.95

INFO:root:Train: 1 [2800/8374 ( 33%)]  Loss: 18.428804 (18.6209)  Loss_kd:  0.342051 (0.3543)  Loss_ce:  1.889238 (1.8722)  Loss_crd: 19.924667 (20.2018)  Loss_g:  2.577804 (2.3297)  Loss_critic:  0.262039 (0.2187)  Time: 1.185s,  387.46/s  (1.196s,  383.73/s)  LR: 0.0025600  Data: 0.016 (0.015)
INFO:root:Train: 1 [2850/8374 ( 34%)]  Loss: 18.344086 (18.6157)  Loss_kd:  0.361118 (0.3542)  Loss_ce:  1.878698 (1.8717)  Loss_crd: 19.807745 (20.1954)  Loss_g:  2.580725 (2.3342)  Loss_critic:  0.263217 (0.2194)  Time: 1.180s,  389.01/s  (1.196s,  383.80/s)  LR: 0.0025600  Data: 0.013 (0.015)
INFO:root:Train: 1 [2900/8374 ( 35%)]  Loss: 18.361786 (18.6103)  Loss_kd:  0.343696 (0.3542)  Loss_ce:  1.894543 (1.8712)  Loss_crd: 19.828663 (20.1889)  Loss_g:  2.606167 (2.3386)  Loss_critic:  0.262107 (0.2202)  Time: 1.182s,  388.35/s  (1.196s,  383.80/s)  LR: 0.0025600  Data: 0.013 (0.015)
INFO:root:Train: 1 [2950/8374 ( 35%)]  Loss: 18.150734 (18.6049)  Loss_kd:  0.354145 (0.3540)  Loss_ce:  1.88

...

INFO:root:Train: 21 [5750/8374 ( 69%)]  Loss:  4.324215 (4.4919)  Loss_kd:  0.211444 (0.2247)  Loss_ce:  1.875620 (1.8309)  Loss_crd:  2.494798 (2.7416)  Loss_g:  2.413131 (2.4311)  Loss_critic:  0.245871 (0.2468)  Time: 1.189s,  386.07/s  (1.196s,  383.63/s)  LR: 0.0020064  Data: 0.013 (0.014)
INFO:root:Train: 21 [5800/8374 ( 69%)]  Loss:  4.628972 (4.4918)  Loss_kd:  0.206575 (0.2246)  Loss_ce:  2.023413 (1.8309)  Loss_crd:  2.698850 (2.7415)  Loss_g:  2.399044 (2.4310)  Loss_critic:  0.244697 (0.2468)  Time: 1.192s,  385.11/s  (1.196s,  383.63/s)  LR: 0.0020064  Data: 0.013 (0.014)
INFO:root:Train: 21 [5850/8374 ( 70%)]  Loss:  4.193893 (4.4921)  Loss_kd:  0.213871 (0.2246)  Loss_ce:  1.646883 (1.8309)  Loss_crd:  2.613886 (2.7418)  Loss_g:  2.420299 (2.4310)  Loss_critic:  0.246893 (0.2468)  Time: 1.189s,  386.14/s  (1.196s,  383.62/s)  LR: 0.0020064  Data: 0.013 (0.014)
INFO:root:Train: 21 [5900/8374 ( 70%)]  Loss:  4.904676 (4.4921)  Loss_kd:  0.230812 (0.2246)  Loss_ce:  1.99499

INFO:root:Train: 21 [7150/8374 ( 85%)]  Loss:  4.802181 (4.4915)  Loss_kd:  0.245006 (0.2245)  Loss_ce:  1.928112 (1.8311)  Loss_crd:  2.977225 (2.7410)  Loss_g:  2.472831 (2.4311)  Loss_critic:  0.249976 (0.2468)  Time: 1.188s,  386.30/s  (1.197s,  383.60/s)  LR: 0.0020064  Data: 0.013 (0.014)
INFO:root:Train: 21 [7200/8374 ( 86%)]  Loss:  4.829387 (4.4917)  Loss_kd:  0.231294 (0.2245)  Loss_ce:  1.921848 (1.8313)  Loss_crd:  3.040691 (2.7410)  Loss_g:  2.436925 (2.4311)  Loss_critic:  0.247051 (0.2468)  Time: 1.250s,  367.09/s  (1.197s,  383.60/s)  LR: 0.0020064  Data: 0.013 (0.014)
INFO:root:Train: 21 [7250/8374 ( 87%)]  Loss:  4.252782 (4.4919)  Loss_kd:  0.211863 (0.2245)  Loss_ce:  1.739096 (1.8314)  Loss_crd:  2.579002 (2.7411)  Loss_g:  2.386210 (2.4310)  Loss_critic:  0.245409 (0.2468)  Time: 1.188s,  386.45/s  (1.197s,  383.59/s)  LR: 0.0020064  Data: 0.013 (0.014)
INFO:root:Train: 21 [7300/8374 ( 87%)]  Loss:  4.512677 (4.4918)  Loss_kd:  0.229753 (0.2245)  Loss_ce:  1.86982

INFO:root:Test: [  20/327]  Time: 0.333s (0.641s,  238.67/s)  Loss:  1.3477 (0.8628)  Acc@1:  72.549 ( 82.602)  Acc@5:  93.464 ( 96.794)
INFO:root:Test: [  30/327]  Time: 0.333s (0.542s,  282.47/s)  Loss:  0.4037 (0.8441)  Acc@1:  95.425 ( 83.112)  Acc@5:  98.693 ( 96.542)
INFO:root:Test: [  40/327]  Time: 0.333s (0.491s,  311.83/s)  Loss:  1.0579 (0.8584)  Acc@1:  80.392 ( 83.501)  Acc@5:  94.118 ( 96.493)
INFO:root:Test: [  50/327]  Time: 0.333s (0.460s,  332.74/s)  Loss:  0.8737 (0.8087)  Acc@1:  86.275 ( 85.031)  Acc@5:  94.771 ( 96.758)
INFO:root:Test: [  60/327]  Time: 0.333s (0.439s,  348.41/s)  Loss:  1.1057 (0.8351)  Acc@1:  71.242 ( 83.853)  Acc@5:  96.732 ( 96.646)
INFO:root:Test: [  70/327]  Time: 0.333s (0.424s,  360.73/s)  Loss:  0.6398 (0.8407)  Acc@1:  91.503 ( 83.697)  Acc@5:  97.386 ( 96.640)
INFO:root:Test: [  80/327]  Time: 0.333s (0.413s,  370.33/s)  Loss:  0.6524 (0.8435)  Acc@1:  90.196 ( 83.394)  Acc@5:  98.039 ( 96.651)
INFO:root:Test: [  90/327]  Time: 0.332s 

INFO:root:Train: 22 [ 700/8374 (  8%)]  Loss:  4.407228 (4.4704)  Loss_kd:  0.235157 (0.2238)  Loss_ce:  1.904516 (1.8208)  Loss_crd:  2.530297 (2.7283)  Loss_g:  2.433170 (2.4320)  Loss_critic:  0.248456 (0.2468)  Time: 1.228s,  373.77/s  (1.203s,  381.70/s)  LR: 0.0019462  Data: 0.013 (0.019)
INFO:root:Train: 22 [ 750/8374 (  9%)]  Loss:  4.570683 (4.4707)  Loss_kd:  0.207803 (0.2238)  Loss_ce:  1.779203 (1.8210)  Loss_crd:  2.927766 (2.7285)  Loss_g:  2.414636 (2.4315)  Loss_critic:  0.244267 (0.2467)  Time: 1.187s,  386.59/s  (1.202s,  381.93/s)  LR: 0.0019462  Data: 0.013 (0.019)
INFO:root:Train: 22 [ 800/8374 ( 10%)]  Loss:  4.465635 (4.4700)  Loss_kd:  0.216127 (0.2236)  Loss_ce:  1.855769 (1.8210)  Loss_crd:  2.689476 (2.7278)  Loss_g:  2.421583 (2.4308)  Loss_critic:  0.245341 (0.2467)  Time: 1.189s,  385.93/s  (1.202s,  381.99/s)  LR: 0.0019462  Data: 0.013 (0.018)
INFO:root:Train: 22 [ 850/8374 ( 10%)]  Loss:  4.988639 (4.4704)  Loss_kd:  0.236432 (0.2237)  Loss_ce:  2.05837

INFO:root:Train: 22 [2100/8374 ( 25%)]  Loss:  4.651918 (4.4717)  Loss_kd:  0.227772 (0.2234)  Loss_ce:  1.926684 (1.8229)  Loss_crd:  2.814976 (2.7278)  Loss_g:  2.454808 (2.4308)  Loss_critic:  0.247543 (0.2467)  Time: 1.188s,  386.22/s  (1.197s,  383.52/s)  LR: 0.0019462  Data: 0.013 (0.015)
INFO:root:Train: 22 [2150/8374 ( 26%)]  Loss:  4.681684 (4.4733)  Loss_kd:  0.231442 (0.2234)  Loss_ce:  1.987818 (1.8239)  Loss_crd:  2.770710 (2.7287)  Loss_g:  2.458569 (2.4306)  Loss_critic:  0.248196 (0.2467)  Time: 1.212s,  378.74/s  (1.197s,  383.54/s)  LR: 0.0019462  Data: 0.013 (0.015)
INFO:root:Train: 22 [2200/8374 ( 26%)]  Loss:  4.406093 (4.4727)  Loss_kd:  0.221709 (0.2234)  Loss_ce:  1.715826 (1.8237)  Loss_crd:  2.784370 (2.7283)  Loss_g:  2.410611 (2.4305)  Loss_critic:  0.247092 (0.2467)  Time: 1.193s,  384.66/s  (1.197s,  383.56/s)  LR: 0.0019462  Data: 0.013 (0.015)
INFO:root:Train: 22 [2250/8374 ( 27%)]  Loss:  4.434633 (4.4736)  Loss_kd:  0.225715 (0.2234)  Loss_ce:  1.86650